In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
import statsmodels.api as sm
from functools import reduce
import scipy.stats as stats
from collections import OrderedDict
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import random

# suppress warning

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns
# avoid scientific digit
pd.options.display.float_format = '{:.8f}'.format

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
unit_df = pd.read_csv('/content/drive/My Drive/X999/unit_df_normalized.csv')

In [ ]:
nta_df = pd.read_csv('/content/drive/My Drive/X999/nta_df_normalized.csv')

In [ ]:
bin_df = pd.read_csv('/content/drive/My Drive/X999/BIN_df_normalized.csv')

In [ ]:
zipcodes_df = pd.read_csv('/content/drive/My Drive/X999/zipcode_df_normalized.csv')

In [ ]:
complaints_columns = ['air_quality', 'animal_issues', 'appliances',  'building_exterior', 'doors_windows', 'electrical_issues', 'elevator_issues',
'floors_stairs','general_complaints', 'graffiti_posting', 'heat_hot_water','homeless_issues', 'noise_complaints','other_issues', 'pest_issues',
'plumbing_issues', 'police_matters', 'public_nuisance','safety_concerns', 'sanitation_issues', 'walls_ceilings', 'total_complaints']

In [ ]:
# for item in complaints_columns:
#   nta_df.rename(columns={item: item + '_nta'}, inplace=True)
#   bin_df.rename(columns={item: item + '_bin'}, inplace=True)
#   zipcodes_df.rename(columns={item: item + '_zipcode'}, inplace=True)

In [ ]:
zipcodes_df.head()

,zipcode,eviction_count_zipcode,eviction_count_unit,eviction_count_building,eviction_count_nta,month_year,year,nta,bin,census_tract,council_district,community_board,longitude,latitude,borough,executed_date,eviction_apartment_number,eviction_address,docket_number,court_index_number,bbl,primary_key,Location,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,numfloors,unitsres,bldgarea,bldgclass,building_type,architectural_style,building_age,building_category,is_condo,is_llc,avg_idr,avg_idr_change,income_change,income_change_ratio,median_income,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white
0,10002,61.00000000,0.00073801,0.60000000,56.72749658,2019-03,2019,Lower East Side,1077523,202.00000000,2.00000000,3.00000000,-73.98286600,40.71138100,MANHATTAN,2019-03-15,05E,630 WATER STREET,115084,014679/18,1002600001,014679/18_115084,Lower East Side,2.63255622,0.10182496,-0.24441651,-0.23014407,-0.35120112,-0.41575982,-0.12829407,-0.45963993,-0.36198386,-0.20618317,-0.28308591,0.54446416,1.63664822,12.00138064,0.10621251,-0.26712625,0.30834208,1.10000168,-0.43745134,-0.39014153,-0.41691376,0.88264038,1.44206581,0.47110497,0.45858918,1.71124008,-0.48218687,1.25805475,0.22376957,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,0.33656949,0.39990329,-0.13447419,-0.08025525,0.31459399,0.37685617,-0.56794790,1.07678865,-0.68257460,2.20956395,-1.80374947,0.02659191,1.40547024,1.27492190,-0.79928722,-0.18133595,-0.95824164,-0.55596050,2.41678554,-0.25158118,-0.28234398,-0.15525459,-0.32955230,0.32955230
1,10009,67.00000000,0.00042988,0.80000000,56.72749658,2018-08,2018,Lower East Side,1078059,20.00000000,2.00000000,3.00000000,-73.97489500,40.72071700,MANHATTAN,2018-08-22,11E,711 FDR DR,112423,015506/17,1003560001,015506/17_112423,Lower East Side,-0.21785384,2.46930182,-0.24441651,-0.23014407,-0.23378550,-0.41575982,0.10719624,-0.45963993,-0.36198386,-0.20618317,-0.28308591,2.03610482,1.79042197,2.86113171,0.36455922,-0.23116852,0.09503932,3.24885496,-0.23214575,0.39008113,-0.41691376,1.11511494,0.98541046,2.68364222,2.18646591,1.20268377,2.07388476,1.12782242,0.42979777,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
2,10009,67.00000000,0.00215633,0.80000000,56.72749658,2023-05,2023,Lower East Side,1083144,28.00000000,2.00000000,3.00000000,-73.97598600,40.72682400,MANHATTAN,2023-05-26,15D,3 HAVEN PLAZA,5780,0300290/22,1003820001,0300290/22_5780,Lower East Side,0.73228284,-0.06728053,-0.07592603,-0.23014407,0.11846136,0.60546195,0.38978462,-0.06594408,0.17064092,-0.20618317,0.08827574,-0.09481040,-0.02177249,-0.18561793,-0.00450751,-0.01542219,-0.07560289,0.17906456,-0.02684016,0.03419009,-0.14815433,0.04264942,0.98541046,2.68364222,2.18646591,1.20268377,2.07388476,1.12782242,0.42979777,-0.82605988,-0.07404125,-0.68750473,3.31408053,0.85138169,-0.35004721,-0.34884320,-0.38793712,-0.79251171,-1.29195986,-1.22414657,0.28882161,-0.63052791,-0.22514595,-1.42864940,-0.86031694,-1.56374299,0.40252163,-2.33941623,0.29181198,-0.53719203,1.02689236,-1.34858768,-1.35328275,-1.00784528,-0.61702397,0.38290781,-0.25158118,1.00217679,-0.28518706,-1.55423998,1.55423998
3,10009,67.00000000,0.00215633,0.80000000,56.72749658,2023-06

In [ ]:
building_columns = ['numfloors', 'unitsres', 'bldgarea', 'bldgclass','building_type', 'architectural_style', "building_age",
                    'building_category', 'is_condo','is_llc']

In [ ]:
# for itme in building_columns:
#   nta_df.rename(columns={itme: itme + '_nta'}, inplace=True)
#   zipcodes_df.rename(columns={itme: itme + '_zipcode'}, inplace=True)

In [ ]:
racial_vars = ['ep_afam', 'ep_hisp', 'ep_asian', 'ep_nhpi', 'ep_twomore', 'ep_otherrace', 'ep_minrty', 'ep_white']
gentrification_vars= ['avg_idr','avg_idr_change','income_change','income_change_ratio']
socialeconomic_vars = ['ep_pov150', 'ep_unemp', 'ep_nohsdp', "median_income",
 'ep_uninsur', 'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', 'ep_hburd',]
svi_vars = ["rpl_theme1", "rpl_theme2", "rpl_theme3", "rpl_theme4", 'rpl_themes']
building_vars = ['numfloors', 'unitsres', 'bldgarea', 'bldgclass','building_type', 'architectural_style', "building_age",
                    'building_category', 'is_condo','is_llc']
complaints_vars = ['air_quality', 'animal_issues', 'appliances',  'building_exterior', 'doors_windows', 'electrical_issues', 'elevator_issues',
'floors_stairs','general_complaints', 'graffiti_posting', 'heat_hot_water','homeless_issues', 'noise_complaints','other_issues', 'pest_issues',
'plumbing_issues', 'police_matters', 'public_nuisance','safety_concerns', 'sanitation_issues', 'walls_ceilings', 'total_complaints']
feature_columns = racial_vars + socialeconomic_vars + svi_vars + building_vars + complaints_vars + gentrification_vars

In [ ]:
# identifier_columns = ['Location', 'primary_key','bbl','court_index_number','docket_number','eviction_address','eviction_apartment_number',
#  'executed_date','borough','zipcode','latitude','longitude','community_board','council_district','census_tract','bin','nta','year','month_year']
# metrics_columns = ['eviction_count_nta','eviction_count_building','eviction_count_unit', 'eviction_count_zipcode']
# building_columns = ['numfloors', 'unitsres', 'bldgarea', 'bldgclass','building_type', 'architectural_style', "building_age",
#                     'building_category', 'is_condo','is_llc']
# complaints_columns = ['air_quality', 'animal_issues', 'appliances',  'building_exterior', 'doors_windows', 'electrical_issues', 'elevator_issues',
# 'floors_stairs','general_complaints', 'graffiti_posting', 'heat_hot_water','homeless_issues', 'noise_complaints','other_issues', 'pest_issues',
# 'plumbing_issues', 'police_matters', 'public_nuisance','safety_concerns', 'sanitation_issues', 'walls_ceilings', 'total_complaints']
# other_features_columns = ['avg_idr','avg_idr_change','income_change','income_change_ratio',
# 'median_income', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4', 'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp',
#  'ep_uninsur', 'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp', 'ep_asian',
#  'ep_nhpi', 'ep_twomore', 'ep_otherrace', 'ep_minrty', 'ep_white']
# all = building_columns + complaints_columns + other_features_columns
# len(all), len(feature_columns)

In [ ]:
set(list(all)) - set(list(feature_columns))

set()

In [ ]:
len(feature_columns)

61

In [ ]:
groups_ordered = OrderedDict([
    ('Racial', racial_vars),
    ('Gentrification Proxies', socialeconomic_vars),
    ('Socialeconomic', gentrification_vars),
    ('SVI', svi_vars),
    ('Building', building_vars),
    ('Complaints', complaints_vars)
])

In [ ]:
results_nta = {}
cumulative_vars = []

In [ ]:
for name, vars in groups_ordered.items():
    cumulative_vars.extend(vars)
    formula = f"np.log(eviction_count_nta + 1) ~ {' + '.join(cumulative_vars)}"
    model = sm.OLS.from_formula(formula, data=nta_df).fit()
    # results = []
    results_nta[f"Model up to {name}"] = {
        "R-squared": model.rsquared,
        "Adj. R-squared": model.rsquared_adj,
        "Added Group": name,
        "New Vars": vars,
        # how r-squares change
        # this metric matters as the determinant for the importances of the factors
        "Delta R-squared": model.rsquared - (results_nta[list(results_nta.keys())[-1]]["R-squared"] if results_nta else 0)
    }

results_df_nta = pd.DataFrame(results_nta).T

In [ ]:
results_df_nta.sort_values("Delta R-squared", ascending=True)

,R-squared,Adj. R-squared,Added Group,New Vars,Delta R-squared
Model up to Complaints,0.77619139,0.77599233,Complaints,"[air_quality, animal_issues, appliances, building_exterior, doors_windows, electrical_issues, elevator_issues, floors_stairs, general_complaints, graffiti_posting, heat_hot_water, homeless_issues, noise_complaints, other_issues, pest_issues, plumbing_issues, police_matters, public_nuisance, safety_concerns, sanitation_issues, walls_ceilings, total_complaints]",0.00182490
Model up to Socialeconomic,0.73312020,0.73302772,Socialeconomic,"[avg_idr, avg_idr_change, income_change, income_change_ratio]",0.00679307
Model up to SVI,0.74381804,0.74370996,SVI,"[rpl_theme1, rpl_theme2, rpl_theme3, rpl_theme4, rpl_themes]",0.01069784
Model up to Building,0.77436649,0.77423728,Building,"[numfloors, unitsres, bldgarea, bldgclass, building_type, architectural_style, building_age, building_category, is_condo, is_llc]",0.03054845
Model up to Gentrification Proxies,0.72632713,0.72624879,Gentrification Proxies,"[ep_pov150, ep_unemp, ep_nohsdp, median_income, ep_uninsur, ep_age65, ep_age17, ep_disabl, ep_limeng, ep_noveh, ep_crowd, ep_hburd]",0.16294278
Model up to Racial,0.56338435,0.56333831,Racial,"[ep_afam, ep_hisp, ep_asian, ep_nhpi, ep_twomore, ep_otherrace, ep_minrty, ep_white]",0.56338435


In [ ]:
results_zipcode = {}
cumulative_vars = []

In [ ]:
for name, vars in groups_ordered.items():
    cumulative_vars.extend(vars)
    formula = f"np.log(eviction_count_zipcode + 1) ~ {' + '.join(cumulative_vars)}"
    model = sm.OLS.from_formula(formula, data=zipcodes_df).fit()
    # results = []
    results_zipcode[f"Model up to {name}"] = {
        "R-squared": model.rsquared,
        "Adj. R-squared": model.rsquared_adj,
        "Added Group": name,
        "New Vars": vars,
        # how r-squares change
        # this metric matters as the determinant for the importances of the factors
        "Delta R-squared": model.rsquared - (results_zipcode[list(results_zipcode.keys())[-1]]["R-squared"] if results_zipcode else 0)
    }

results_df_zipcode = pd.DataFrame(results_zipcode).T

In [ ]:
results_df_zipcode.sort_values("Delta R-squared", ascending=True)

,R-squared,Adj. R-squared,Added Group,New Vars,Delta R-squared
Model up to Complaints,0.85508955,0.85496066,Complaints,"[air_quality, animal_issues, appliances, building_exterior, doors_windows, electrical_issues, elevator_issues, floors_stairs, general_complaints, graffiti_posting, heat_hot_water, homeless_issues, noise_complaints, other_issues, pest_issues, plumbing_issues, police_matters, public_nuisance, safety_concerns, sanitation_issues, walls_ceilings, total_complaints]",0.00212774
Model up to Socialeconomic,0.78664750,0.78657356,Socialeconomic,"[avg_idr, avg_idr_change, income_change, income_change_ratio]",0.00712766
Model up to Building,0.85296181,0.85287761,Building,"[numfloors, unitsres, bldgarea, bldgclass, building_type, architectural_style, building_age, building_category, is_condo, is_llc]",0.02530432
Model up to SVI,0.82765749,0.82758478,SVI,"[rpl_theme1, rpl_theme2, rpl_theme3, rpl_theme4, rpl_themes]",0.04101000
Model up to Gentrification Proxies,0.77951984,0.77945672,Gentrification Proxies,"[ep_pov150, ep_unemp, ep_nohsdp, median_income, ep_uninsur, ep_age65, ep_age17, ep_disabl, ep_limeng, ep_noveh, ep_crowd, ep_hburd]",0.14613486
Model up to Racial,0.63338498,0.63334632,Racial,"[ep_afam, ep_hisp, ep_asian, ep_nhpi, ep_twomore, ep_otherrace, ep_minrty, ep_white]",0.63338498


In [ ]:
# groups = {
#     "Gentrification": ["avg_idr", "avg_change_abs", 'income_change', 'income_change_ratio'],
#     "Economic": ['ep_nohsdp', 'ep_age65','ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh', 'ep_crowd', "ep_pov150", "ep_unemp", "ep_hburd", 'ep_uninsur', 'median_income'],
#     "SVI": ["rpl_theme1", "rpl_theme2", "rpl_theme3", "rpl_theme4"],
#     "Building": ['yearbuilt', 'numfloors', 'unitsres', 'bldgarea', 'building_type','is_condo'],
#     "Complaints": ['air_quality', 'animal_issues', \
#        'appliances', 'building_exterior', 'doors_windows', 'electrical_issues', \
#        'elevator_issues', 'floors_stairs', 'general_complaints', \
#        'graffiti_posting', 'heat_hot_water', 'homeless_issues', \
#        'noise_complaints', 'other_issues', 'pest_issues', 'plumbing_issues', \
#        'police_matters', 'public_nuisance', 'safety_concerns', 'sanitation_issues', 'walls_ceilings', 'total_complaints'],
#     "Racial": ['ep_afam', 'ep_hisp', 'ep_asian','ep_nhpi', 'ep_twomore', 'ep_otherrace', 'ep_minrty', 'ep_white',],
# }

# # groups_df = pd.DataFrame(groups)
# # groups_df

In [ ]:
results_bin = {}
cumulative_vars = []

In [ ]:
for name, vars in groups_ordered.items():
    cumulative_vars.extend(vars)
    formula = f"np.log(eviction_count_building + 1) ~ {' + '.join(cumulative_vars)}"
    model = sm.OLS.from_formula(formula, data=bin_df).fit()
    # results = []
    results_bin[f"Model up to {name}"] = {
        "R-squared": model.rsquared,
        "Adj. R-squared": model.rsquared_adj,
        "Added Group": name,
        "New Vars": vars,
        # how r-squares change
        # this metric matters as the determinant for the importances of the factors
        "Delta R-squared": model.rsquared - (results_bin[list(results_bin.keys())[-1]]["R-squared"] if results_bin else 0)
    }
results_df_bin = pd.DataFrame(results_bin).T

In [ ]:
results_df_bin.sort_values("Delta R-squared", ascending=True)

,R-squared,Adj. R-squared,Added Group,New Vars,Delta R-squared
Model up to SVI,0.13817362,0.13781002,SVI,"[rpl_theme1, rpl_theme2, rpl_theme3, rpl_theme4, rpl_themes]",0.01240799
Model up to Socialeconomic,0.12576564,0.12546268,Socialeconomic,"[avg_idr, avg_idr_change, income_change, income_change_ratio]",0.01266257
Model up to Gentrification Proxies,0.11310306,0.11284919,Gentrification Proxies,"[ep_pov150, ep_unemp, ep_nohsdp, median_income, ep_uninsur, ep_age65, ep_age17, ep_disabl, ep_limeng, ep_noveh, ep_crowd, ep_hburd]",0.03572906
Model up to Racial,0.07737400,0.07727672,Racial,"[ep_afam, ep_hisp, ep_asian, ep_nhpi, ep_twomore, ep_otherrace, ep_minrty, ep_white]",0.07737400
Model up to Building,0.34547313,0.34509830,Building,"[numfloors, unitsres, bldgarea, bldgclass, building_type, architectural_style, building_age, building_category, is_condo, is_llc]",0.20729951
Model up to Complaints,0.56620826,0.56582243,Complaints,"[air_quality, animal_issues, appliances, building_exterior, doors_windows, electrical_issues, elevator_issues, floors_stairs, general_complaints, graffiti_posting, heat_hot_water, homeless_issues, noise_complaints, other_issues, pest_issues, plumbing_issues, police_matters, public_nuisance, safety_concerns, sanitation_issues, walls_ceilings, total_complaints]",0.22073513


In [ ]:
results_unit = {}
cumulative_vars = []

In [ ]:
for name, vars in groups_ordered.items():
    cumulative_vars.extend(vars)
    formula = f"np.log(eviction_count_unit + 1) ~ {' + '.join(cumulative_vars)}"
    model = sm.OLS.from_formula(formula, data=unit_df).fit()
    # results = []
    results_unit[f"Model up to {name}"] = {
        "R-squared": model.rsquared,
        "Adj. R-squared": model.rsquared_adj,
        "Added Group": name,
        "New Vars": vars,
        # how r-squares change
        # this metric matters as the determinant for the importances of the factors
        "Delta R-squared": model.rsquared - (results_unit[list(results_unit.keys())[-1]]["R-squared"] if results_unit else 0)
    }

results_df_unit = pd.DataFrame(results_unit).T

In [ ]:
results_df_unit.sort_values("Delta R-squared", ascending=True)

,R-squared,Adj. R-squared,Added Group,New Vars,Delta R-squared
Model up to Socialeconomic,0.17506500,0.17477912,Socialeconomic,"[avg_idr, avg_idr_change, income_change, income_change_ratio]",0.00239958
Model up to Complaints,0.43026768,0.42976095,Complaints,"[air_quality, animal_issues, appliances, building_exterior, doors_windows, electrical_issues, elevator_issues, floors_stairs, general_complaints, graffiti_posting, heat_hot_water, homeless_issues, noise_complaints, other_issues, pest_issues, plumbing_issues, police_matters, public_nuisance, safety_concerns, sanitation_issues, walls_ceilings, total_complaints]",0.00677651
Model up to SVI,0.18522985,0.18488610,SVI,"[rpl_theme1, rpl_theme2, rpl_theme3, rpl_theme4, rpl_themes]",0.01016486
Model up to Gentrification Proxies,0.17266541,0.17242859,Gentrification Proxies,"[ep_pov150, ep_unemp, ep_nohsdp, median_income, ep_uninsur, ep_age65, ep_age17, ep_disabl, ep_limeng, ep_noveh, ep_crowd, ep_hburd]",0.08129726
Model up to Racial,0.09136815,0.09127234,Racial,"[ep_afam, ep_hisp, ep_asian, ep_nhpi, ep_twomore, ep_otherrace, ep_minrty, ep_white]",0.09136815
Model up to Building,0.42349117,0.42316102,Building,"[numfloors, unitsres, bldgarea, bldgclass, building_type, architectural_style, building_age, building_category, is_condo, is_llc]",0.23826132


In [ ]:
# Median Income has the most impact as a single variable:
median_income_model_unit = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ median_income",
    data=unit_df
).fit()
median_income_model_building = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ median_income",
    data=bin_df
).fit()
median_income_model_zipcode = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ median_income",
    data=zipcodes_df
).fit()
median_income_model_nta = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ median_income",
    data=nta_df
).fit()
print("nta mi: ", median_income_model_unit.rsquared) # 0.0000521
print("building mi: ", median_income_model_building.rsquared)
print("zipcode mi: ", median_income_model_zipcode.rsquared)
print("unit mi: ", median_income_model_nta.rsquared)

nta mi:  5.2100268646326775e-05
building mi:  0.053678282127582744
zipcode mi:  0.42951613739772776
unit mi:  0.43835699663267946


In [ ]:
# Racial:
racial_model_unit = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ ep_afam + ep_hisp + ep_asian + ep_nhpi + ep_twomore + ep_otherrace + ep_minrty + ep_white",
    data=unit_df
).fit()
racial_model_building = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ ep_afam + ep_hisp + ep_asian + ep_nhpi + ep_twomore + ep_otherrace + ep_minrty + ep_white",
    data=bin_df
).fit()
racial_model_zipcode = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ ep_afam + ep_hisp + ep_asian + ep_nhpi + ep_twomore + ep_otherrace + ep_minrty + ep_white",
    data=zipcodes_df
).fit()
racial_model_nta = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ ep_afam + ep_hisp + ep_asian + ep_nhpi + ep_twomore + ep_otherrace + ep_minrty + ep_white",
    data=nta_df
).fit()
print("nta racial: ", racial_model_unit.rsquared)
print("building racial: ", racial_model_building.rsquared)
print("zipcode racial: ", racial_model_zipcode.rsquared)
print("unit racial: ", racial_model_nta.rsquared)

nta racial:  0.09136815144754451
building racial:  0.07737400074354561
zipcode racial:  0.6333849800812975
unit racial:  0.5633843480912144


In [ ]:
# Black + Hispanic combo could capture the most varainces in eviction data
bh_model_unit = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ ep_afam + ep_hisp",
    data=unit_df
).fit()
bh_model_building = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ ep_afam + ep_hisp",
    data= bin_df
).fit()
bh_model_zipcode = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ ep_afam + ep_hisp",
    data= zipcodes_df
).fit()
bh_model_nta = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ ep_afam + ep_hisp",
    data= nta_df
).fit()
print("unit black + hispanic: ", bh_model_unit.rsquared)
print("building black + hispanic: ", bh_model_building.rsquared)
print("zipcode black + hispanic: ", bh_model_zipcode.rsquared)
print("nta black + hispanic: ", bh_model_nta.rsquared)

unit black + hispanic:  0.051104617979182754
building black + hispanic:  0.07131988570693026
zipcode black + hispanic:  0.6068924203645827
nta black + hispanic:  0.5346146584901373


In [ ]:
# Complaints:
complaints_model_unit = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ air_quality + animal_issues + appliances + building_exterior + doors_windows + electrical_issues + \
       elevator_issues + floors_stairs + general_complaints + graffiti_posting + heat_hot_water + homeless_issues + noise_complaints + other_issues + \
       pest_issues + plumbing_issues + police_matters + public_nuisance + safety_concerns + sanitation_issues + walls_ceilings + total_complaints",
    data= unit_df
).fit()
complaints_model_building = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ air_quality + animal_issues + appliances + building_exterior + doors_windows + electrical_issues + \
       elevator_issues + floors_stairs + general_complaints + graffiti_posting + heat_hot_water + homeless_issues + noise_complaints + other_issues + \
       pest_issues + plumbing_issues + police_matters + public_nuisance + safety_concerns + sanitation_issues + walls_ceilings + total_complaints",
    data= bin_df
).fit()
complaints_model_zipcode = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ air_quality + animal_issues + appliances + building_exterior + doors_windows + electrical_issues + \
       elevator_issues + floors_stairs + general_complaints + graffiti_posting + heat_hot_water + homeless_issues + noise_complaints + other_issues + \
       pest_issues + plumbing_issues + police_matters + public_nuisance + safety_concerns + sanitation_issues + walls_ceilings + total_complaints",
    data= zipcodes_df
).fit()
complaints_model_nta = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ air_quality + animal_issues + appliances + building_exterior + doors_windows + electrical_issues + \
       elevator_issues + floors_stairs + general_complaints + graffiti_posting + heat_hot_water + homeless_issues + noise_complaints + other_issues + \
       pest_issues + plumbing_issues + police_matters + public_nuisance + safety_concerns + sanitation_issues + walls_ceilings + total_complaints",
    data= nta_df
).fit()
print("unit complaints: ", complaints_model_unit.rsquared)
print("building complaints: ", complaints_model_building.rsquared)
print("zipcode complaints: ", complaints_model_zipcode.rsquared)
print("nta complaints: ", complaints_model_nta.rsquared)

unit complaints:  0.05286198744561166
building complaints:  0.41587940635221765
zipcode complaints:  0.06254053181099672
nta complaints:  0.04983855311932972


In [ ]:
# Buildings:
building_model_unit = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ numfloors + unitsres + bldgarea + bldgclass + building_type + architectural_style + building_age + \
                    building_category + is_condo + is_llc",
    data= unit_df
).fit()
building_model_building = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ numfloors + unitsres + bldgarea + bldgclass + building_type + architectural_style + building_age + \
                    building_category + is_condo + is_llc",
    data= bin_df
).fit()
building_model_zipcode = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ numfloors + unitsres + bldgarea + bldgclass + building_type + architectural_style + building_age + \
                    building_category + is_condo + is_llc",
    data= zipcodes_df
).fit()
building_model_nta = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ numfloors + unitsres + bldgarea + bldgclass + building_type + architectural_style + building_age + \
                    building_category + is_condo + is_llc",
    data= nta_df
).fit()
print("unit building chars.: ", building_model_unit.rsquared)
print("building building chars.: ", building_model_building.rsquared)
print("zipcode building chars.: ", building_model_zipcode.rsquared)
print("nta building chars.: ", building_model_nta.rsquared)

unit building chars.:  0.3856494060345501
building building chars.:  0.22368068916036754
zipcode building chars.:  0.26733743765576146
nta building chars.:  0.2230725180713442


In [ ]:
# SVI:
svi_model_unit = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ rpl_theme1 + rpl_theme2 + rpl_theme3 + rpl_theme4 + rpl_themes",
    data= unit_df
).fit()
svi_model_building = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ rpl_theme1 + rpl_theme2 + rpl_theme3 + rpl_theme4 + rpl_themes",
    data= bin_df
).fit()
svi_model_zipcode = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ rpl_theme1 + rpl_theme2 + rpl_theme3 + rpl_theme4 + rpl_themes",
    data= zipcodes_df
).fit()
svi_model_nta = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ rpl_theme1 + rpl_theme2 + rpl_theme3 + rpl_theme4 + rpl_themes",
    data=  nta_df
).fit()
print("unit SVI scores: ", svi_model_unit.rsquared)
print("building SVI scores: ", svi_model_building.rsquared)
print("zipcode SVI scores: ", svi_model_zipcode.rsquared)
print("nta SVI scores: ", svi_model_nta.rsquared)

unit SVI scores:  0.06878913655783447
building SVI scores:  0.08822638008547201
zipcode SVI scores:  0.633576958031258
nta SVI scores:  0.42619949980729255


In [ ]:
# Gentrification:
gentrification_model_unit = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ avg_idr +avg_idr_change + income_change + income_change_ratio",
    data= unit_df
).fit()
gentrification_model_building = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ avg_idr +avg_idr_change + income_change + income_change_ratio",
    data= bin_df
).fit()
gentrification_model_zipcode = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ avg_idr +avg_idr_change + income_change + income_change_ratio",
    data= zipcodes_df
).fit()
gentrification_model_nta = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ avg_idr +avg_idr_change + income_change + income_change_ratio",
    data= nta_df
).fit()
print("unit gentrification proxies: ", gentrification_model_unit.rsquared)
print("building gentrification proxies: ", gentrification_model_building.rsquared)
print("zipcode gentrification proxies: ", gentrification_model_zipcode.rsquared)
print("nta gentrification proxies: ", gentrification_model_nta.rsquared)

unit gentrification proxies:  0.02619140566237166
building gentrification proxies:  0.0347744461138354
zipcode gentrification proxies:  0.21247363099446337
nta gentrification proxies:  0.25947886963969535


In [ ]:
# socialeconomic:
socialeconomic_model_unit = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ ep_nohsdp + ep_age65 + ep_age17 + ep_disabl + ep_limeng + ep_noveh + ep_crowd + ep_pov150 + ep_unemp + ep_hburd + ep_uninsur + median_income",
    data= unit_df
).fit()
socialeconomic_model_building = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ ep_nohsdp + ep_age65 + ep_age17 + ep_disabl + ep_limeng + ep_noveh + ep_crowd + ep_pov150 + ep_unemp + ep_hburd + ep_uninsur + median_income",
    data= bin_df
).fit()
socialeconomic_model_zipcode = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ ep_nohsdp + ep_age65 + ep_age17 + ep_disabl + ep_limeng + ep_noveh + ep_crowd + ep_pov150 + ep_unemp + ep_hburd + ep_uninsur + median_income",
    data= zipcodes_df
).fit()
socialeconomic_model_nta = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ ep_nohsdp + ep_age65 + ep_age17 + ep_disabl + ep_limeng + ep_noveh + ep_crowd + ep_pov150 + ep_unemp + ep_hburd + ep_uninsur + median_income",
    data= nta_df
).fit()
print("unit economic vars: ", socialeconomic_model_unit.rsquared)
print("building economic vars: ", socialeconomic_model_building.rsquared)
print("zipcode economic vars: ", socialeconomic_model_zipcode.rsquared)
print("nta economic vars: ", socialeconomic_model_nta.rsquared)

unit economic vars:  0.15218470353054092
building economic vars:  0.10780795181804759
zipcode economic vars:  0.7158429440503489
nta economic vars:  0.6954211290314223


In [ ]:
data = {
    "unit": {
        "socialeconomic": socialeconomic_model_unit.rsquared,
        "racial": racial_model_unit.rsquared,
        "gentrification": gentrification_model_unit.rsquared,
        "svi": svi_model_unit.rsquared,
        "building": building_model_unit.rsquared,
        "complaints": complaints_model_unit.rsquared,
        "median_income": median_income_model_unit.rsquared,
        "bh": bh_model_unit.rsquared,
    },
    "building": {
        "socialeconomic": socialeconomic_model_building.rsquared,
        "racial": racial_model_building.rsquared,
        "gentrification": gentrification_model_building.rsquared,
        "svi": svi_model_building.rsquared,
        "building": building_model_building.rsquared,
        "complaints": complaints_model_building.rsquared,
        "median_income": median_income_model_building.rsquared,
        "bh": bh_model_building.rsquared,
    },
    "zipcode": {
        "socialeconomic": socialeconomic_model_zipcode.rsquared,
        "racial": racial_model_zipcode.rsquared,
        "gentrification": gentrification_model_zipcode.rsquared,
        "svi": svi_model_building.rsquared,
        "building": building_model_zipcode.rsquared,
        "complaints": complaints_model_zipcode.rsquared,
        "median_income": median_income_model_zipcode.rsquared,
        "bh": bh_model_zipcode.rsquared,
    },
    "nta": {
        "socialeconomic": socialeconomic_model_nta.rsquared,
        "racial": racial_model_nta.rsquared,
        "gentrification": gentrification_model_nta.rsquared,
        "svi": svi_model_nta.rsquared,
        "building": building_model_nta.rsquared,
        "complaints": complaints_model_nta.rsquared,
        "median_income": median_income_model_nta.rsquared,
        "bh": bh_model_nta.rsquared,
    },
}

data_df = pd.DataFrame(data)
data_df

,unit,building,zipcode,nta
socialeconomic,0.15218470,0.10780795,0.71584294,0.69542113
racial,0.09136815,0.07737400,0.63338498,0.56338435
gentrification,0.02619141,0.03477445,0.21247363,0.25947887
svi,0.06878914,0.08822638,0.08822638,0.42619950
building,0.38564941,0.22368069,0.26733744,0.22307252
complaints,0.05286199,0.41587941,0.06254053,0.04983855
median_income,0.00005210,0.05367828,0.42951614,0.43835700
bh,0.05110462,0.07131989,0.60689242,0.53461466


In [ ]:
data_df.T

,socialeconomic,racial,gentrification,svi,building,complaints,median_income,bh
unit,0.15218470,0.09136815,0.02619141,0.06878914,0.38564941,0.05286199,0.00005210,0.05110462
building,0.10780795,0.07737400,0.03477445,0.08822638,0.22368069,0.41587941,0.05367828,0.07131989
zipcode,0.71584294,0.63338498,0.21247363,0.08822638,0.26733744,0.06254053,0.42951614,0.60689242
nta,0.69542113,0.56338435,0.25947887,0.42619950,0.22307252,0.04983855,0.43835700,0.53461466


In [ ]:
features = {
    "socialeconomic": socialeconomic_model_unit.params,
    "racial": racial_model_unit.params,
    "gentrification": gentrification_model_unit.params,
    "svi": svi_model_unit.params,
    "building": building_model_unit.params,
    "complaints": complaints_model_unit.params,
    "median_income": median_income_model_unit.params,
    "bh": bh_model_unit.params,
}

# features_df = pd.DataFrame(features)
# features_df

In [ ]:
# combinaton 1
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ avg_idr+ avg_idr_change + ep_afam + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150",
    data= zipcodes_df
).fit()
print(final_model.summary())

                                  OLS Regression Results                                  
Dep. Variable:     np.log(eviction_count_nta + 1)   R-squared:                       0.656
Model:                                        OLS   Adj. R-squared:                  0.656
Method:                             Least Squares   F-statistic:                 1.579e+04
Date:                            Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                    20:21:49   Log-Likelihood:                -31998.
No. Observations:                           66395   AIC:                         6.401e+04
Df Residuals:                               66386   BIC:                         6.410e+04
Df Model:                                       8                                         
Covariance Type:                        nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
-------

In [ ]:
# yearbuilt, numfloors, unitsres, bldgarea, building_type, is_condo
# the best combo
# 13 features capture 72% of eviction data variances
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_zipcode + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data=zipcodes_df
).fit()
print(final_model.summary())

                                    OLS Regression Results                                    
Dep. Variable:     np.log(eviction_count_zipcode + 1)   R-squared:                       0.766
Model:                                            OLS   Adj. R-squared:                  0.765
Method:                                 Least Squares   F-statistic:                 1.667e+04
Date:                                Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                        20:21:49   Log-Likelihood:                -39244.
No. Observations:                               66395   AIC:                         7.852e+04
Df Residuals:                                   66381   BIC:                         7.864e+04
Df Model:                                          13                                         
Covariance Type:                            nonrobust                                         
                       coef    std err          t 

In [ ]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_nta + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data=nta_df
).fit()
print(final_model.summary())

                                  OLS Regression Results                                  
Dep. Variable:     np.log(eviction_count_nta + 1)   R-squared:                       0.693
Model:                                        OLS   Adj. R-squared:                  0.693
Method:                             Least Squares   F-statistic:                 1.154e+04
Date:                            Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                    20:21:50   Log-Likelihood:                -28153.
No. Observations:                           66395   AIC:                         5.633e+04
Df Residuals:                               66381   BIC:                         5.646e+04
Df Model:                                      13                                         
Covariance Type:                        nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----

In [ ]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_building + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data= bin_df
).fit()
print(final_model.summary())

                                     OLS Regression Results                                    
Dep. Variable:     np.log(eviction_count_building + 1)   R-squared:                       0.345
Model:                                             OLS   Adj. R-squared:                  0.345
Method:                                  Least Squares   F-statistic:                     2687.
Date:                                 Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                         20:21:50   Log-Likelihood:                -26603.
No. Observations:                                66395   AIC:                         5.323e+04
Df Residuals:                                    66381   BIC:                         5.336e+04
Df Model:                                           13                                         
Covariance Type:                             nonrobust                                         
                       coef    std err  

In [ ]:
final_model = sm.OLS.from_formula(
    "np.log(eviction_count_unit + 1) ~ avg_idr + avg_idr_change + ep_afam + income_change + median_income + ep_hisp + ep_hburd + is_llc + ep_pov150 + rpl_themes + unitsres + building_age + total_complaints",
    data= unit_df
).fit()
print(final_model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     np.log(eviction_count_unit + 1)   R-squared:                       0.163
Model:                                         OLS   Adj. R-squared:                  0.163
Method:                              Least Squares   F-statistic:                     996.6
Date:                             Tue, 10 Jun 2025   Prob (F-statistic):               0.00
Time:                                     20:21:50   Log-Likelihood:                 88709.
No. Observations:                            66395   AIC:                        -1.774e+05
Df Residuals:                                66381   BIC:                        -1.773e+05
Df Model:                                       13                                         
Covariance Type:                         nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.